### get svg points array from csv

In [1]:
csv_name = "./isolated.csv"

In [2]:
import numpy as np

csv_path = csv_name


def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=",")
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs


paths = read_csv(csv_path)

### convert the svg to image

In [3]:
import svgwrite
import os

os.environ["path"] += r";C:\Program Files\GTK3-Runtime Win64\bin"
import cairosvg


def polylines2svg(
    paths_XYs,
    svg_path,
    colours=["yellow"],
):
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)
    # Create a new SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile="tiny", shape_rendering="crispEdges")
    group = dwg.g()
    for i, path in enumerate(paths_XYs):
        path_data = []
        c = colours[i % len(colours)]
        for XY in path:
            path_data.append(("M", (XY[0, 0], XY[0, 1])))
            for j in range(1, len(XY)):
                path_data.append(("L", (XY[j, 0], XY[j, 1])))
            # if not np.allclose(XY[0], XY[-1]):
            #     path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill="none", stroke=c, stroke_width=2))
    dwg.add(group)
    dwg.save()
    png_path = svg_path.replace(".svg", ".png")
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(
        url=svg_path,
        write_to=png_path,
        parent_width=W,
        parent_height=H,
        output_width=fact * W,
        output_height=fact * H,
        background_color="white",
    )
    return


polylines2svg(paths, "test.svg")

In [4]:
import numpy as np
print(np.__version__)

2.0.1


### import opencv and initialization

In [6]:
import numpy as np
import cv2  # Only for displaying results, not for shape detection

def detect_shapes(image_path):
    # Load the image
    img = cv2.imread(image_path)

    # Convert to HSV color space
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define color thresholds (adjust as needed)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Create masks
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Find contours (connected components)
    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.04 * cv2.arcLength(cnt, True), True)

        # Calculate area to filter out small contours
        area = cv2.contourArea(cnt)
        if area < 500:  # Adjust threshold as needed
            continue

        if len(approx) == 4:
            shape = "square"
            # Draw a green square border
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        elif len(approx) > 4 and area > 1000:  # Filter out small circles
            shape = "circle"
            # Draw a blue circle border
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            center = (int(x), int(y))
            radius = int(radius)
            cv2.circle(img, center, radius, (255, 0, 0), 2)
        else:
            shape = "unknown"

        # Draw label
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.putText(img, shape, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.namedWindow('Shapes', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Shapes', 600, 600)
    cv2.imshow("Shapes", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
image_path = "test.png"
detect_shapes(image_path)
